In [12]:
import numpy as np

In [13]:
import math

In [14]:
import matplotlib.pyplot as plt

In [15]:
import sympy

In [16]:
from sympy import *

In [17]:
import matplotlib.animation as animation

In [18]:
import random

In [19]:
import torch

In [20]:
# ランダムな値を生成する関数
def generate_random_values(n):
    return torch.rand(n)

# ランダムな値を生成
num_random_values = 1000  # ランダムな値の数を指定
random_values = generate_random_values(num_random_values)

# 最も近い値を見つける関数
def find_closest_value(given_value, random_values):
    # 与えられた値とランダム値の差の絶対値を計算
    differences = torch.abs(random_values - given_value)
    # 最も小さい差を持つインデックスを取得
    closest_index = torch.argmin(differences)
    # 最も近い値を取得
    closest_value = random_values[closest_index]
    return closest_value.item()

# 実験で与えられた値
given_value = 0.7  # 例として0.7とします

# 最も近い値を見つける
closest_value = find_closest_value(given_value, random_values)

# 結果を表示
print(f"与えられた値: {given_value}")
print(f"最も近いランダムな値: {closest_value}")

与えられた値: 0.7
最も近いランダムな値: 0.7008095383644104


In [21]:
# ランダムな値を生成する関数
def generate_random_values(n):
    return torch.rand(n)

# 最も近い値を見つける関数
def find_closest_value(given_value, random_values):
    # 与えられた値とランダム値の差の絶対値を計算
    differences = torch.abs(random_values - given_value)
    # 最も小さい差を持つインデックスを取得
    closest_index = torch.argmin(differences)
    # 最も近い値を取得
    closest_value = random_values[closest_index]
    return closest_value.item()

# 実験で与えられた値（複数）
given_values = [0.5, 0.7, 0.9]  # 例として3つの値

# ランダムな値を生成
num_random_values = 1000  # ランダムな値の数を指定
random_values = generate_random_values(num_random_values)

# 各与えられた値に対して最も近い値を見つける
for given_value in given_values:
    closest_value = find_closest_value(given_value, random_values)
    # 結果を表示
    print(f"与えられた値: {given_value}")
    print(f"最も近いランダムな値: {closest_value}")


与えられた値: 0.5
最も近いランダムな値: 0.50029456615448
与えられた値: 0.7
最も近いランダムな値: 0.7000223398208618
与えられた値: 0.9
最も近いランダムな値: 0.9012129902839661


In [22]:
# プロットの設定
x = np.linspace(0, 360, 100)

# 単位ベクトルの生成
k = np.array(random.sample(range(1, 100), 3))  # 例としてランダムな単位ベクトルを使用
kn = np.linalg.norm(k)  # 生成したベクトルの大きさ
n = k / kn

# 単位ベクトルに垂直な単位ベクトルの生成
def generate_perpendicular_unit_vector(n):
    """
    入力された単位ベクトル n に垂直な単位ベクトルを生成する
    """
    if np.all(n == 0):
        raise ValueError("The input vector n should not be the zero vector.")
    
    # 任意のベクトル w を選択
    if abs(n[0]) < abs(n[1]) and abs(n[0]) < abs(n[2]):
        w = np.array([1, 0, 0])
    elif abs(n[1]) < abs(n[0]) and abs(n[1]) < abs(n[2]):
        w = np.array([0, 1, 0])
    else:
        w = np.array([0, 0, 1])
    
    # n と w の外積を計算して、n に垂直なベクトルを求める
    perp_vec = np.cross(n, w)
    
    # 垂直なベクトルを正規化して単位ベクトルにする
    perp_unit_vec = perp_vec / np.linalg.norm(perp_vec)
    
    return perp_unit_vec

# sに書き換えてる
s = generate_perpendicular_unit_vector(n)

# thetaの書き換え
theta = np.radians(x)

# ロドリゲスの回転公式×磁場ベクトル
def rodrigues_rotation(n, s, theta):
    h = (s[:, np.newaxis] * np.cos(theta) +
         np.cross(n, s)[:, np.newaxis] * np.sin(theta) +
         n[:, np.newaxis] * np.dot(n, s) * (1 - np.cos(theta)))
    return h

# hに書き換えてる
h = rodrigues_rotation(n, s, theta)

# 原子を回転させるための行列
def rotation_matrix():
    identity_matrix = np.identity(3)
    matrices = []
    for i in range(3):
        kx = 0
        ky = 0
        kz = 1
        R = np.array([[0, -kz, ky], [kz, 0, -kx], [-ky, kx, 0]])
        t = 120
        theta = np.radians(t * i)
        rot_mat = identity_matrix + np.sin(theta) * R + (1 - np.cos(theta)) * np.dot(R, R)
        matrices.append(rot_mat)
    return matrices

# Rに書き換えてる
R = rotation_matrix()

# i = 0, 1, 2 の場合の rot_mat を計算して表示
for i in range(3):
    XX, XY, YY, YZ, ZZ, XZ = [random.uniform(1e-4, 1e-2) for _ in range(6)]
    K = np.array([[XX, XY, XZ], [XY, YY, YZ], [XZ, YZ, ZZ]])
    
    # hの各ベクトルを使ってKを計算
    K_rot_values = []
    for j in range(len(h[0])):
        h_vec = np.array([h[0][j], h[1][j], h[2][j]])
        K_rot = np.dot(h_vec.T, np.dot(R[i], np.dot(K, np.dot(np.linalg.inv(R[i]), h_vec))))
        K_rot_values.append(K_rot)
    
    # 各角度に対するK_rotの値を取得し、表示
    print(f"n: ({n[0]}, {n[1]}, {n[2]}), s: ({s[0]}, {s[1]}, {s[2]}), XX: {XX}, YY: {YY}, ZZ: {ZZ}, XY: {XY}, YZ: {YZ}, XZ: {XZ}")
    print(f"i = {i}")
    for angle, K_rot_value in zip(x, K_rot_values):
        print(f"角度: {angle}, K_rot: {K_rot_value}")

n: (0.07178071525883852, 0.2563596973529947, 0.96391246204726), s: (0.0, 0.9664053651964906, -0.25702270350970496), XX: 0.005537055342147401, YY: 0.005303421274869466, ZZ: 0.003354871780717629, XY: 0.007302154814790584, YZ: 0.0089194304162898, XZ: 0.00045067036807136415
i = 0
角度: 0.0, K_rot: 0.000743737690638733
角度: 3.6363636363636362, K_rot: -3.993659320158574e-05
角度: 7.2727272727272725, K_rot: -0.0007750033984930483
角度: 10.909090909090908, K_rot: -0.0014496352251747772
角度: 14.545454545454545, K_rot: -0.0020529769946148585
角度: 18.18181818181818, K_rot: -0.0025753207118890007
角度: 21.818181818181817, K_rot: -0.003008261671052949
角度: 25.454545454545453, K_rot: -0.003344833690007323
角度: 29.09090909090909, K_rot: -0.0035796211989753157
角度: 32.72727272727273, K_rot: -0.003708846379047648
角度: 36.36363636363636, K_rot: -0.003730429948703007
角度: 40.0, K_rot: -0.0036440246202260797
角度: 43.63636363636363, K_rot: -0.003451020687696893
角度: 47.27272727272727, K_rot: -0.003154523656638627
角度: 50.909